In [ ]:
from io import BytesIO
from tqdm import tqdm
import scipy.stats
import plotly
import plotly.express as px
plotly.io.templates.default = "plotly_dark"

from kaliset import *

In [ ]:
def get_high_freq_strict(k: np.ndarray, verbose: bool = False) -> float:
    if k.ndim > 2:
        k = k[:, :, 0]
    #k = np.clip(k, 0, k.mean())
    k = (k - k.mean())
    #k = np.clip(k, -1, 10)
    #k = (k > 0).astype(np.int)
    if verbose:
        px.imshow(k, title="grid").show()
    k = np.fft.fft2(k).real.sum(axis=1)
    k = k[:k.shape[0]//2]
    k = np.abs(k)
    k = k / k.max()
    k *= (np.linspace(0, 1, k.shape[0]) ** 2)
    if verbose:
        #print((k * 100).astype(int))
        px.line(y=k, title="high freqs").show()
    k = np.abs(k)
    return k.mean()

def get_high_freq(k: np.ndarray, verbose: bool = False) -> float:
    if k.ndim > 2:
        k = k[:, :, 0]
    #k = np.clip(k, 0, k.mean())
    k = (k - k.mean())
    #k = np.clip(k, -1, 10)
    #k = (k > 0).astype(np.int)
    if verbose:
        px.imshow(k, title="grid").show()
    k = np.fft.fft2(k).real.sum(axis=1)
    k = k[:k.shape[0]//2]
    k = np.abs(k)
    #k = k / k.max()
    k *= (np.linspace(0, 1, k.shape[0]) ** 2)
    if verbose:
        #print((k * 100).astype(int))
        px.line(y=k, title="high freqs").show()
    k = np.abs(k)
    return k.mean()

def get_entropy(k: np.ndarray, verbose: bool = False) -> float:
    if k.ndim > 2:
        k = k[:, :, 0]
    k = k.flatten().round(3)
    k = np.clip(k - k.mean(), -1, 1)
    hist, _ = np.histogram(k, bins=1000)
    hist = hist[1:-1]
    hist = np.clip(hist - 1, 0, 1)
    if verbose:
        px.line(y=hist, title="histogram").show()
    return hist.mean()
    #return scipy.stats.entropy(hist)

def get_compression_ratio(data: np.ndarray, color_scale: float = 100, verbose: bool = False):
    img = ndarray_to_image(data, color_scale=color_scale)
    fp = BytesIO()
    img.save(fp, "png")
    memory_size = img.size[0] * img.size[1] * 3
    compress_size = fp.tell()
    return compress_size / memory_size

def get_edges(k: np.ndarray, verbose: bool = False) -> float:
    if k.ndim > 2:
        k = k[:, :, 0]

    conv_weights = [
        [-1, -1, -1],
        [-1, 8, -1],
        [-1, -1, -1],
    ]
    conv = scipy.ndimage.convolve(k, conv_weights)#print(get_compression_ratio(data))
    conv = np.clip(conv.flatten(), 0, 1)
    if verbose:
        px.line(y=conv).show()
    return conv.mean()

def get_freq_entropy(k: np.ndarray, verbose: bool = False) -> float:
    if k.ndim > 2:
        k = k[:, :, 0]
    #k = np.clip(k, 0, k.mean())
    k = (k - k.mean())
    #k = np.clip(k, -1, 1)
    #k = (k > 0).astype(np.int)
    if verbose:
        px.imshow(k, title="grid").show()
    k = np.fft.fft(k.reshape((-1,))).real
    #k = k / k.max()
    k = k[2:k.shape[0]//2]
    k, _ = np.histogram(k, bins=32)
    #k = np.abs(k)
    #k = (k / k.max() * 100).astype(np.uint8) 
    if verbose:
        #print((k * 100).astype(int))
        px.line(y=k, title="quantized freqs").show()
    k = scipy.stats.entropy(k)
    return k

def get_interesting(k: np.ndarray, verbose: bool = False) -> float:
    #high_freq = get_high_freq(k, verbose=verbose)
    #entropy = get_entropy(k, verbose=verbose)
    edges = get_edges(k, verbose=verbose)
    compression = get_compression_ratio(k, verbose=verbose)
    #return entropy * high_freq
    return edges * compression

In [ ]:
for param in (
    [.86, 0.22],
    [.49, 0.26],
    [.72, .72],
    [.72, .33],
    [.52, .4],
    [.89, .23],
    [.52, .92],
    [.93, .57],
):
    #space = KaliSpace()#position=(.5, -.5), scale=.01)
    space = KaliSpace(position=(.5, -.5), scale=.05)
    kset = KaliSet(param, 11)
    print("\nparam:", param)
    kset.display(space, size=[200, 200], color_scale=100)
    print(get_high_freq(kset(space.uv((32, 32)))[:, :, 0], verbose=True))
    

In [ ]:
d = np.asarray([1, 0, 1, 0])
np.fft.fft(d).real

In [ ]:
def sample_params(space, size, func, param_min=(0, 0), param_max=(1, 1)):
    result = np.zeros([size[1]+1, size[0]+1])
    for y in tqdm(range(size[1]+1)):
        for x in range(size[0]+1):
            param = [
                (x / size[0]) * (param_max[0] - param_min[0]) + param_min[0], 
                (y / size[1]) * (param_max[1] - param_min[1]) + param_min[1], 
                .8
            ]
            kset = KaliSet(param, 11)
            k = kset(space.uv((32, 32), dimensions=len(param)))
            #k = k[:, :, 0]
            result[y, x] = func(k)
            #if result[y, x] > 100:
            #    kset.display(space, (64, 64), color_scale=100)
    return result

def sample_param_plot(space, size, func, param_min=(0, 0), param_max=(1, 1)):
    result = sample_params(space, size, func, param_min, param_max)
    labels = {
        "x": f"{param_min[0]} - {param_max[0]}",
        "y": f"{param_min[1]} - {param_max[1]}",
        "color": "interestingness",
        #"x": [str((x / size[0]) * (param_max[0] - param_min[0]) + param_min[0]) for x in range(size[0]+1)],
        #"y": [str((x / size[1]) * (param_max[1] - param_min[1]) + param_min[1]) for x in range(size[1]+1)],
    }
    px.imshow(np.clip(result, 0, 50), labels=labels).show()#, hovertext=labels)

In [ ]:
space = KaliSpace(position=(.5, .5, .5), scale=.1)
#space = KaliSpace()
sample_param_plot(space, (100, 100), get_compression_ratio, param_min=(0., 0.), param_max=(2., 2.))#.round()
#sample_param_plot(space, (30, 30), get_high_freq)

In [ ]:

space = KaliSpace(position=(1, 1), scale=1.)
kset = KaliSet([0.5, 0.5], 11)
data = kset(space.uv((128, 128)))
print(get_edges(data))
#ndarray_to_image(conv, color_scale=100)
#px.imshow(np.clip(result, 8, 50))

In [ ]:

kset = KaliSet([0.97, 0.97], 11)
kset = KaliSet([0.86, 0.32], 11)
#kset = KaliSet([0.905, 0.952], 11)
kset = KaliSet([0.95, 0.78], 11)
#kset = KaliSet([0.72, 0.59], 11)
#kset = KaliSet([0.651, 0.562], 11)
#kset = KaliSet([0.45, 0.06], 11)
kset.display(space, size=(128, 128), color_scale=100)
print(get_interesting(kset(space.uv((32, 32)))[:, :, 0], verbose=True))

In [ ]:
for a in np.linspace(0.5, 1, 5):
    space = KaliSpace()#position=(.5, -.5), scale=.01)
    kset = KaliSet([a, .5], 11)
    k = kset(space.uv((32, 32)))
    
    k = k[:, :, 0]
    #value = get_high_freq(k)
    #if value < 10:
    #    continue
    #print(value)
    
    k = (k - k.mean())
    k = (k > 0).astype(np.int)
    px.imshow(k).show()
    k = np.fft.fft(k.reshape((-1,))).real
    k = np.abs(k)
    print(k)
    print(k[k.shape[0]//2:].sum())
    # print(k.max())
    #k = (k - k.mean())
    #k = (k > 0).astype(np.int)
    #for l in k.tolist():
    #    print(l)
    #k = k[:, :, 0].reshape((-1,))
    #k = np.fft.fft(k.reshape((-1,))).real
    #k = np.abs(k)
    #k = scipy.stats.entropy(k.reshape((-1,)))
    #print(k[5:].mean())#axis=0))
    print(kset.param)
    kset.display(space, size=[128, 128], color_scale=100)


In [ ]:
class KaliSet2(KaliSet):
    def acc_start(self, space: np.ndarray) -> Any:
        return np.ones(space.shape, dtype=space.dtype)

    def accumulate(self, acc: Any, space: np.ndarray, dot_product: np.ndarray):
        acc = np.min(acc, space)

    def acc_end(self, acc: Any, space: np.ndarray) -> Any:
        return acc# / self.iterations
KaliSet2([.5, .5], 11).img(space, size=[200, 200], color_scale=100)

In [ ]:
help(np.fft)

In [ ]:
help(PIL.Image.fromarray)

In [ ]:

#uv = np.linspace([[-1, -1]], [[1, 1]], 5, axis=2)
uv = get_uv(10, 10)
param = np.asarray([.5, .5])
k = kali(uv, param, 7)#.round(2)
px.imshow(np.clip(k[:,:,0], 0, 50))#(rgb)


In [ ]:
#img = PIL.Image.new("RGB", (32, 32))
kk2 = kk.copy()
kk2.k.apply_along_axis
#kk.reshape((-1, ))
#img
#PIL.Image.MODES

In [ ]:
dir(PIL.ImageMode)
#help(PIL.ImageMode)

In [ ]:
c = np.asarray([[1,2], [3,4]])
np.append(c, [[0], [0]], axis=-1)

In [ ]:
k = kali(uv, param, 30)#.round(2)
#rgb = np.apply_along_axis(lambda c: np.append(c, [0]), -1, k)
k[:,:,0]
px.imshow(np.clip(k[:,:,0], 0, 50))#(rgb)
#from scipy.io import ndimage
#ndimage.save

In [ ]:
[n for n in dir(np.ndarray) if "type" in n]

In [ ]:
help(np.apply_along_axis)

In [ ]:
np.linspace([0, 0], [10, 20]).reshape([10, 5, 2])